# Importing the libraries

In [28]:
import pandas as pd
import numpy as np
from datetime import datetime,time
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
# Set the maximum number of displayed columns
pd.set_option('display.max_columns',None) # to display all the columns
# Set the maximum number of displayed rows
pd.set_option('display.max_rows',20)

# Loading the Datasets

In [3]:
in_time = pd.read_csv("in_time.csv")
out_time = pd.read_csv("out_time.csv")
general = pd.read_csv("general_data.csv")
manager_survey = pd.read_csv("manager_survey_data.csv")
employee_survey = pd.read_csv("employee_survey_data.csv")

In [5]:
# Let we take the intime and outtime dataset
in_time.shape

(4410, 262)

In [6]:
in_time.isnull().sum()

Unnamed: 0       0
01-01-2015    4410
02-01-2015     209
05-01-2015     206
06-01-2015     228
              ... 
25-12-2015    4410
28-12-2015     234
29-12-2015     230
30-12-2015     265
31-12-2015     213
Length: 262, dtype: int64

In [7]:
# in the same way check outtime null values
out_time.isnull().sum()

Unnamed: 0       0
01-01-2015    4410
02-01-2015     209
05-01-2015     206
06-01-2015     228
              ... 
25-12-2015    4410
28-12-2015     234
29-12-2015     230
30-12-2015     265
31-12-2015     213
Length: 262, dtype: int64

Here some columns 4410 rows value has nan value it may be holidays so we can drop that columns it cannot affect the outcome(naked eye observation)

In [8]:
#drop the entire columns with the null values of 4410
in_time = in_time[[col for col in in_time.columns if in_time[col].isnull().sum() < 4410]]
out_time = out_time[[col for col in out_time.columns if out_time[col].isnull().sum() < 4410]]

In [9]:
# To check each employee with nun values 
#observation - This may be they took leave for that day so we can add this value as a leave
in_time['Holiday_taken'] = in_time.isnull().sum(axis=1)
out_time['Holiday_taken'] = out_time.isnull().sum(axis=1)

In [29]:
# Rename the column names in both dataset
in_time.rename(columns={"Unnamed: 0":"EmployeeID"},inplace=True)
out_time.rename(columns={"Unnamed: 0":"EmployeeID"},inplace=True)

In [30]:
in_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Columns: 251 entries, EmployeeID to Holiday_taken
dtypes: int64(2), object(249)
memory usage: 8.4+ MB


In [31]:
out_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Columns: 251 entries, EmployeeID to Holiday_taken
dtypes: int64(2), object(249)
memory usage: 8.4+ MB


# Finding Over time Worked Candidates

In [42]:
clear = in_time.drop(columns = ['EmployeeID','Holiday_taken'])
list_of_columns = list(clear.columns)
y = in_time[['EmployeeID','Holiday_taken']]

In [34]:
def avg_working_time(in_time,out_time,list_of_columns):
    columns = []
    for i in list_of_columns:
        in_date = pd.to_datetime(in_time[i])
        out_date = pd.to_datetime(out_time[i])
        
        columns.append(out_date-in_date)
        
    avg_df = pd.concat(columns,axis=1,keys=list_of_columns)
    avg_df['Avg_Working_Time']=avg_df[list_of_columns].mean(axis=1)
    return avg_df['Avg_Working_Time']

In [37]:
def Overtime_Working(avg):
    avg['Avg_Working_Time']=pd.to_timedelta(avg['Avg_Working_Time'])
    ot_threshold = pd.Timedelta('8 hours')
    avg['Overtime_Working'] = avg['Avg_Working_Time'] > ot_threshold
    avg['Overtime_Working'] = avg['Overtime_Working'].map({True:'Yes',False:'No'})
    
    return avg

In [41]:
avg = pd.DataFrame(avg_working_time(in_time,out_time,list_of_columns))
df_ot = pd.DataFrame(Overtime_Working(avg))
df_ot

,Avg_Working_Time,Overtime_Working
0,0 days 07:22:26.896551724,No
1,0 days 07:43:03.813559322,No
2,0 days 07:00:48.595041322,No
3,0 days 07:11:36.510638297,No
4,0 days 08:00:22.530612244,Yes
...,...,...
4405,0 days 08:31:20,Yes
4406,0 days 06:05:35.103734439,No
4407,0 days 07:42:24.935064935,No
4408,0 days 09:29:32.365145228,Yes


In [43]:
df_ot_merge = pd.concat([y,df_ot],axis=1)

In [45]:
df_ot_merge.drop(columns = ['Avg_Working_Time'])

,EmployeeID,Holiday_taken,Overtime_Working
0,1,17,No
1,2,13,No
2,3,7,No
3,4,14,No
4,5,4,Yes
...,...,...,...
4405,4406,6,Yes
4406,4407,8,No
4407,4408,18,No
4408,4409,8,Yes


In [46]:
# Merge surveys with df_ot_merge
df_survey = manager_survey.merge(employee_survey,on='EmployeeID')
df_survey_merge = df_survey.merge(df_ot_merge,on='EmployeeID')
df_final = df_survey_merge.merge(general,on='EmployeeID')
df_final

,EmployeeID,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Holiday_taken,Avg_Working_Time,Overtime_Working,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,1,3,3,3.0,4.0,2.0,17,0 days 07:22:26.896551724,No,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,Healthcare Representative,Married,131160,1.0,Y,11,8,0,1.0,6,1,0,0
1,2,2,4,3.0,2.0,4.0,13,0 days 07:43:03.813559322,No,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,1,Research Scientist,Single,41890,0.0,Y,23,8,1,6.0,3,5,1,4
2,3,3,3,2.0,2.0,1.0,7,0 days 07:00:48.595041322,No,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,4,Sales Executive,Married,193280,1.0,Y,15,8,3,5.0,2,5,0,3
3,4,2,3,4.0,4.0,3.0,14,0 days 07:11:36.510638297,No,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,3,Human Resources,Married,83210,3.0,Y,11,8,3,13.0,5,8,7,5
4,5,3,3,4.0,1.0,3.0,4,0 days 08:00:22.530612244,Yes,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,1,Sales Executive,Single,23420,4.0,Y,12,8,2,9.0,2,6,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,3,3,4.0,1.0,3.0,6,0 days 08:31:20,Yes,42,No,Travel_Rarely,Research & Development,5,4,Medical,1,Female,1,Research Scientist,Single,60290,3.0,Y,17,8,1,10.0,5,3,0,2
4406,4407,2,3,4.0,4.0,3.0,8,0 days 06:05:35.103734439,No,29,No,Travel_Rarely,Research & Development,2,4,Medical,1,Male,1,Laboratory Technician,Divorced,26790,2.0,Y,15,8,0,10.0,2,3,0,2
4407,4408,3,4,1.0,3.0,3.0,18,0 days 07:42:24.935064935,No,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,1,Male,2,Sales Executive,Married,37020,0.0,Y,20,8,0,5.0,4,4,1,2
4408,4409,2,3,4.0,1.0,3.0,8,0 days 09:29:32.365145228,Yes,42,No,Travel_Rarely,Sales,18,2,Medical,1,Male,1,Laboratory Technician,Divorced,23980,0.0,Y,14,8,1,10.0,2,9,7,8


In [47]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 0 to 4409
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype          
---  ------                   --------------  -----          
 0   EmployeeID               4410 non-null   int64          
 1   JobInvolvement           4410 non-null   int64          
 2   PerformanceRating        4410 non-null   int64          
 3   EnvironmentSatisfaction  4385 non-null   float64        
 4   JobSatisfaction          4390 non-null   float64        
 5   WorkLifeBalance          4372 non-null   float64        
 6   Holiday_taken            4410 non-null   int64          
 7   Avg_Working_Time         4410 non-null   timedelta64[ns]
 8   Overtime_Working         4410 non-null   object         
 9   Age                      4410 non-null   int64          
 10  Attrition                4410 non-null   object         
 11  BusinessTravel           4410 non-null   object         
 12  Department          

In [54]:
# Finding the missing values
null = df_final.isnull().sum()
columns_with_null = null[null > 0]
columns_with_null

EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
NumCompaniesWorked         19
TotalWorkingYears           9
dtype: int64

In [57]:
# Filling the Nan values with specific values
# Filling Work life Balance
median_worklife = np.round(df_final['WorkLifeBalance'].median())
df_final['WorkLifeBalance'] = df_final['WorkLifeBalance'].fillna(median_worklife)

# Filling Environment Satisfaction
median_environment = np.round(df_final['EnvironmentSatisfaction'].median())
df_final['EnvironmentSatisfaction'] = df_final['EnvironmentSatisfaction'].fillna(median_environment)

# Filling Job Satisfaction
median_job = np.round(df_final['JobSatisfaction'].median())
df_final['JobSatisfaction'].fillna(median_job,inplace=True)

#Filling NumCompaniesWorked 
df_final['NumCompaniesWorked'].fillna(0,inplace=True)

#Filling Total Working Years
df_final['TotalWorkingYears'] = df_final.groupby('YearsAtCompany')['TotalWorkingYears'].transform(lambda x:x.fillna(x.mean()))


In [58]:
df_final

,EmployeeID,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Holiday_taken,Avg_Working_Time,Overtime_Working,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,1,3,3,3.0,4.0,2.0,17,0 days 07:22:26.896551724,No,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,Healthcare Representative,Married,131160,1.0,Y,11,8,0,1.000000,6,1,0,0
1,2,2,4,3.0,2.0,4.0,13,0 days 07:43:03.813559322,No,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,1,Research Scientist,Single,41890,0.0,Y,23,8,1,6.000000,3,5,1,4
2,3,3,3,2.0,2.0,1.0,7,0 days 07:00:48.595041322,No,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,4,Sales Executive,Married,193280,1.0,Y,15,8,3,5.000000,2,5,0,3
3,4,2,3,4.0,4.0,3.0,14,0 days 07:11:36.510638297,No,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,3,Human Resources,Married,83210,3.0,Y,11,8,3,13.000000,5,8,7,5
4,5,3,3,4.0,1.0,3.0,4,0 days 08:00:22.530612244,Yes,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,1,Sales Executive,Single,23420,4.0,Y,12,8,2,9.000000,2,6,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,3,3,4.0,1.0,3.0,6,0 days 08:31:20,Yes,42,No,Travel_Rarely,Research & Development,5,4,Medical,1,Female,1,Research Scientist,Single,60290,3.0,Y,17,8,1,10.000000,5,3,0,2
4406,4407,2,3,4.0,4.0,3.0,8,0 days 06:05:35.103734439,No,29,No,Travel_Rarely,Research & Development,2,4,Medical,1,Male,1,Laboratory Technician,Divorced,26790,2.0,Y,15,8,0,10.000000,2,3,0,2
4407,4408,3,4,1.0,3.0,3.0,18,0 days 07:42:24.935064935,No,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,1,Male,2,Sales Executive,Married,37020,0.0,Y,20,8,0,5.000000,4,4,1,2
4408,4409,2,3,4.0,1.0,3.0,8,0 days 09:29:32.365145228,Yes,42,No,Travel_Rarely,Sales,18,2,Medical,1,Male,1,Laboratory Technician,Divorced,23980,0.0,Y,14,8,1,10.000000,2,9,7,8


In [60]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 0 to 4409
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype          
---  ------                   --------------  -----          
 0   EmployeeID               4410 non-null   int64          
 1   JobInvolvement           4410 non-null   int64          
 2   PerformanceRating        4410 non-null   int64          
 3   EnvironmentSatisfaction  4410 non-null   float64        
 4   JobSatisfaction          4410 non-null   float64        
 5   WorkLifeBalance          4410 non-null   float64        
 6   Holiday_taken            4410 non-null   int64          
 7   Avg_Working_Time         4410 non-null   timedelta64[ns]
 8   Overtime_Working         4410 non-null   object         
 9   Age                      4410 non-null   int64          
 10  Attrition                4410 non-null   object         
 11  BusinessTravel           4410 non-null   object         
 12  Department          

In [61]:
df_final.head()

,EmployeeID,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Holiday_taken,Avg_Working_Time,Overtime_Working,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,1,3,3,3.0,4.0,2.0,17,0 days 07:22:26.896551724,No,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,Healthcare Representative,Married,131160,1.0,Y,11,8,0,1.0,6,1,0,0
1,2,2,4,3.0,2.0,4.0,13,0 days 07:43:03.813559322,No,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,1,Research Scientist,Single,41890,0.0,Y,23,8,1,6.0,3,5,1,4
2,3,3,3,2.0,2.0,1.0,7,0 days 07:00:48.595041322,No,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,4,Sales Executive,Married,193280,1.0,Y,15,8,3,5.0,2,5,0,3
3,4,2,3,4.0,4.0,3.0,14,0 days 07:11:36.510638297,No,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,3,Human Resources,Married,83210,3.0,Y,11,8,3,13.0,5,8,7,5
4,5,3,3,4.0,1.0,3.0,4,0 days 08:00:22.530612244,Yes,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,1,Sales Executive,Single,23420,4.0,Y,12,8,2,9.0,2,6,0,4


In [62]:
# Drop the unwanted columns
df_final = df_final.drop(columns = ['Avg_Working_Time','EmployeeCount','Over18','StandardHours','TrainingTimesLastYear'])

In [63]:
#Changing the datatypes
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 0 to 4409
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   JobInvolvement           4410 non-null   int64  
 2   PerformanceRating        4410 non-null   int64  
 3   EnvironmentSatisfaction  4410 non-null   float64
 4   JobSatisfaction          4410 non-null   float64
 5   WorkLifeBalance          4410 non-null   float64
 6   Holiday_taken            4410 non-null   int64  
 7   Overtime_Working         4410 non-null   object 
 8   Age                      4410 non-null   int64  
 9   Attrition                4410 non-null   object 
 10  BusinessTravel           4410 non-null   object 
 11  Department               4410 non-null   object 
 12  DistanceFromHome         4410 non-null   int64  
 13  Education                4410 non-null   int64  
 14  EducationField          

In [64]:
df_final['EnvironmentSatisfaction']=df_final['EnvironmentSatisfaction'].astype(float).astype(int)
df_final['JobSatisfaction']=df_final['JobSatisfaction'].astype(float).astype(int)
df_final['WorkLifeBalance']=df_final['WorkLifeBalance'].astype(float).astype(int)
df_final['NumCompaniesWorked']=df_final['NumCompaniesWorked'].astype(float).astype(int)
df_final['TotalWorkingYears']=df_final['TotalWorkingYears'].astype(float).astype(int)

In [66]:
#Change the cleaned dataframe into csv
df_final.to_csv("Cleaned_Attrition_CSV",index=False)